In [1]:
### Import modules

In [2]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from imutils import paths
import numpy as np
import argparse
import os
import pickle

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# construct the argument parser and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-d", "--dataset", type=str, default="3scenes",
#	help="path to directory containing the '3scenes' dataset")
#args = vars(ap.parse_args())
dataset = "C:/Users/UTILISATEUR/Sandbox/catfaces/dataset"

In [4]:
print("[INFO] loading images...")
imagePaths = paths.list_images(dataset)
data = []
labels = []

[INFO] loading images...


In [5]:
# loop over our input images
for imagePath in imagePaths:
	# load the input image from disk, resize it to 32x32 pixels, scale
	# the pixel intensities to the range [0, 1], and then update our
	# images list
	image = Image.open(imagePath)
	image = np.array(image.resize((224, 224))) / 255.0
	data.append(image)
 
	# extract the class label from the file path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

In [6]:
# encode the labels, converting them from strings to integers
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [7]:
# perform a training and testing split, using 75% of the data for
# training and 25% for evaluation
(trainX, testX, trainY, testY) = train_test_split(np.array(data),np.array(labels), test_size=0.10)
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY, test_size=0.20)

In [8]:
# 13/08/2019 definition architecture reseau de neurones : Catson v2
model = Sequential()

model.add(Conv2D(8, (3, 3), padding="same", input_shape=(224, 224, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(16, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(9))
model.add(Activation("softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 8)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
activation_2 (Activation)    (None, 112, 112, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 32)        4640      
__________

In [9]:
# train the model using the Adam optimizer
print("[INFO] training network...")
opt = Adam(lr=1e-3, decay=1e-3 / 50)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(valX, valY), epochs=15, batch_size=32)

[INFO] training network...
Train on 2353 samples, validate on 589 samples
Epoch 1/15
2353/2353 [==============================] - 105s 45ms/step - loss: 0.9246 - acc: 0.6906 - val_loss: 0.4441 - val_acc: 0.8557
Epoch 2/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.3414 - acc: 0.8870 - val_loss: 0.2404 - val_acc: 0.9151
Epoch 3/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.2078 - acc: 0.9316 - val_loss: 0.2207 - val_acc: 0.9491
Epoch 4/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.1682 - acc: 0.9490 - val_loss: 0.1218 - val_acc: 0.9576
Epoch 5/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.0811 - acc: 0.9690 - val_loss: 0.1603 - val_acc: 0.9576
Epoch 6/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.0591 - acc: 0.9779 - val_loss: 0.1033 - val_acc: 0.9762
Epoch 7/15
2353/2353 [==============================] - 5s 2ms/step - loss: 0.0441 - acc: 0.9843 - val_loss: 0.1428 - v

In [10]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
             precision    recall  f1-score   support

     bagguy       0.99      0.99      0.99        82
      felix       1.00      0.94      0.97        48
     gritus       0.80      1.00      0.89        24
    grokado       0.75      1.00      0.86         6
      kipik       1.00      0.50      0.67         2
     mikado       1.00      0.79      0.88        29
      nocat       1.00      1.00      1.00       131
        ota       1.00      0.67      0.80         3
    triknik       0.40      1.00      0.57         2

avg / total       0.97      0.96      0.96       327



In [11]:
# saving neural model
model.save('cat_neural_model.h5')

with open('cat_model_architecture.json', 'w') as f:
    f.write(model.to_json())

f = open("cat_labels.bin", "wb")
f.write(pickle.dumps(lb))
f.close()